In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

from src.entities import GSA

from typing import Union

In [2]:
# Define corridor

corridor = {"MAD": {
                "CIU": {
                    "COR": {
                        "SEV": {
                            "CAD": {}
                        },
                        "PGE": {
                            "ANT": {
                                "GRA": {},
                                "MAL": {}
                                    }
                                }
                            }
                        }
                    }
            }

In [3]:
# Get lines from corridor

def get_lines(corridor: dict, path: Union[list, None]=None) -> list:
    """
    Get all the lines in the corridor
    
    Args:
        corridor (dict): dictionary with the corridor structure
        path (list, optional): list of nodes
    
    Returns:
        list of lines
    """
    if path is None:
        path = []

    lines = []
    for node, child in corridor.items():
        new_path = path + [node]
        if not child:  # If the node has no children, it is a leaf
            lines.append(new_path)
        else:
            lines.extend(get_lines(child, new_path))  # If the node has children, we call the function recursively

    return lines

get_lines(corridor)

[['MAD', 'CIU', 'COR', 'SEV', 'CAD'],
 ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'GRA'],
 ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'MAL']]

In [4]:
def sample_line(lines: list) -> list:
    """
    Sample a random line from the list of lines
    
    Args:
        lines (list): list of lines
    
    Returns:
        list: random line
    """
    return lines[np.random.randint(len(lines))] 

def sample_route(line: list) -> list:
    """
    Sample a random route from line
    
    Args:
        line (list): list of stations
        
    Returns:
        list: random route
    """
    return line[np.random.randint(0, len(line)-1):]

lines = get_lines(corridor)

line = sample_line(lines)
print(f"Sampled line: {line}")

# Sample a random route from line (at least two stations)
route = sample_route(line)
print(f"Sampled route: {route}")

Sampled line: ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'GRA']
Sampled route: ['ANT', 'GRA']


In [5]:
def get_timetable(route: list) -> dict:
    """
    Generate random timetable for route r
    
    Args:
        route (list): list of stations
    
    Returns:
        dict: timetable
    """
    timetable = {}
    AT = np.random.randint(0, 24*60)
    DT = AT
    for i, sta in enumerate(route):
        if i == 0 or i == len(route)-1:
            timetable[sta] = (AT, AT)
        else:
            timetable[sta] = (AT, DT)
            
        AT += np.random.randint(30, 120)
        DT = AT + np.random.randint(2, 8)
        
    return timetable

get_timetable(route)

{'ANT': (350, 350), 'GRA': (448, 448)}

In [16]:
# Generate random requested timetable in corridor for a day t

# Number of services:
n_services = 10

# 1) Generate random timetable
schedule = {i: get_timetable(sample_route(sample_line(lines))) for i in range(1, n_services+1)}
schedule

{1: {'COR': (948, 948),
  'PGE': (988, 990),
  'ANT': (1105, 1110),
  'GRA': (1188, 1188)},
 2: {'CIU': (111, 111),
  'COR': (196, 203),
  'PGE': (228, 230),
  'ANT': (277, 279),
  'MAL': (314, 314)},
 3: {'CIU': (1265, 1265),
  'COR': (1365, 1371),
  'PGE': (1432, 1436),
  'ANT': (1524, 1529),
  'MAL': (1633, 1633)},
 4: {'SEV': (91, 91), 'CAD': (161, 161)},
 5: {'COR': (73, 73), 'PGE': (170, 176), 'ANT': (237, 240), 'MAL': (306, 306)},
 6: {'ANT': (1083, 1083), 'MAL': (1121, 1121)},
 7: {'SEV': (74, 74), 'CAD': (165, 165)},
 8: {'COR': (1180, 1180),
  'PGE': (1216, 1218),
  'ANT': (1329, 1333),
  'MAL': (1420, 1420)},
 9: {'CIU': (191, 191),
  'COR': (308, 314),
  'SEV': (390, 396),
  'CAD': (455, 455)},
 10: {'CIU': (952, 952),
  'COR': (984, 991),
  'PGE': (1029, 1034),
  'ANT': (1078, 1083),
  'MAL': (1138, 1138)}}

In [29]:
schedule = {1: {'MAD': (0, 0), 'BAR': (148, 152), 'FIG': (180, 180)},
            2: {'MAD': (8, 8), 'ZAR': (28, 30), 'BAR': (165, 167), 'FIG': (210, 210)},
            3: {'MAD': (30, 30), 'BAR': (180, 182), 'FIG': (225, 225)}}

In [30]:
def get_real_vars(schedule):
    real_vars = []
    for service in schedule:
        for i, stop in enumerate(schedule[service]):
            if i == 0 or i == len(schedule[service]) - 1:
                real_vars.append(schedule[service][stop][0])
            else: 
                real_vars.append(schedule[service][stop][0])
                real_vars.append(schedule[service][stop][1])
                
    return real_vars

real_vars = get_real_vars(schedule)
real_vars

[0, 148, 152, 180, 8, 28, 30, 165, 167, 210, 30, 180, 182, 225]

In [31]:
def get_real_boundaries(real_vars):
    real_boundaries = []
    for rv in real_vars:
        real_boundaries.append((rv-10, rv+10))
    
    return real_boundaries
    
real_boundaries = get_real_boundaries(real_vars)
real_boundaries

[(-10, 10),
 (138, 158),
 (142, 162),
 (170, 190),
 (-2, 18),
 (18, 38),
 (20, 40),
 (155, 175),
 (157, 177),
 (200, 220),
 (20, 40),
 (170, 190),
 (172, 192),
 (215, 235)]

In [32]:
def update_schedule(solution):
    """
    Update schedule with the solution
    
    Args:
        solution (list): solution
    """
    s_idx = 0
    for service in schedule:
        for i, stop in enumerate(schedule[service]):
            if i == 0 or i == len(schedule[service]) - 1:
                schedule[service][stop] = (solution[s_idx], solution[s_idx])
                s_idx += 1
            else:
                schedule[service][stop] = (solution[s_idx], solution[s_idx+1])
                s_idx += 2
                
update_schedule(real_vars)
schedule

{1: {'MAD': (0, 0), 'BAR': (148, 152), 'FIG': (180, 180)},
 2: {'MAD': (8, 8), 'ZAR': (28, 30), 'BAR': (165, 167), 'FIG': (210, 210)},
 3: {'MAD': (30, 30), 'BAR': (180, 182), 'FIG': (225, 225)}}

In [33]:
def is_feasible(solution, security_gap=10):
    """
    Check if the solution is feasible
    
    Args:
        solution (dict): solution
        security_gap (int, optional): security gap
        
    Returns:
        bool: True if the solution is feasible, False otherwise
    """
    S_i = np.array(solution["discrete"])
    update_schedule(solution["real"])
    security_array = []
    
    # Get conflicts between services
    for service in schedule:
        service_sec_arr = []  # Build security matrix for service (columns are stops, rows are other services)
        for service_k in schedule:
          service_sec_row = [] 
          for stop in schedule[service]:
            if service_k == service or stop not in schedule[service_k]:
              service_sec_row.append(0)
              continue
        
            if abs(schedule[service][stop][1] - schedule[service_k][stop][1]) < security_gap:
              service_sec_row.append(1)
            else:
              service_sec_row.append(0)
          service_sec_arr.append(service_sec_row)
        
        security_array.append(np.array(service_sec_arr))
    
    # Get array of security conflicts. If there is a conflict, the dot product will be different from zero
    if not S_i.dot(np.array([np.sum(S_i.dot(ssa)) for ssa in security_array])):
        return True
    return False

"""
schedule = {1: {'MAD': (0, 0), 'BAR': (148, 152), 'FIG': (180, 180)},
            2: {'MAD': (8, 8), 'ZAR': (28, 30), 'BAR': (165, 167), 'FIG': (210, 210)},
            3: {'MAD': (30, 30), 'BAR': (180, 182), 'FIG': (225, 225)}}

solution = {"real": [0, 148, 152, 180, 8, 28, 30, 165, 167, 210, 30, 180, 182, 225], "discrete": [1, 1, 1]}

is_feasible(solution)
"""

'\nschedule = {1: {\'MAD\': (0, 0), \'BAR\': (148, 152), \'FIG\': (180, 180)},\n            2: {\'MAD\': (8, 8), \'ZAR\': (28, 30), \'BAR\': (165, 167), \'FIG\': (210, 210)},\n            3: {\'MAD\': (30, 30), \'BAR\': (180, 182), \'FIG\': (225, 225)}}\n\nsolution = {"real": [0, 148, 152, 180, 8, 28, 30, 165, 167, 210, 30, 180, 182, 225], "discrete": [1, 1, 1]}\n\nis_feasible(solution)\n'

In [34]:
def get_num_trains(solution):
    scheduled_trains = np.sum(solution["discrete"])
    print("Requested number of trains: ", len(solution["discrete"]))
    print("Scheduled trains: ", scheduled_trains)
    return scheduled_trains, 0

In [35]:
boundaries = {'real': real_boundaries, 'discrete': [(0, 1) for _ in range(len(schedule))]}

gsa_algo = GSA(objective_function=get_num_trains,
               is_feasible=is_feasible,
               r_dim=len(real_vars),
               d_dim=len(schedule),
               boundaries=boundaries)

In [36]:
gsa_algo.set_seed(seed=28)

training_history = gsa_algo.optimize(population_size=5,
                                     iters=300,
                                     chaotic_constant=False,
                                     repair_solution=True)

Initializing positions of the individuals in the population...
Positions of the individuals in the population successfully initialized!!
GSA is optimizing  "get_num_trains"
Requested number of trains:  3
Scheduled trains:  0
Requested number of trains:  3
Scheduled trains:  1
Requested number of trains:  3
Scheduled trains:  1
Requested number of trains:  3
Scheduled trains:  2
Requested number of trains:  3
Scheduled trains:  1
####################################################################################################
REPAIRING SOLUTION... 
####################################################################################################
####################################################################################################
SOLUTION SUCCESSFULLY REPAIRED!!
####################################################################################################
####################################################################################################
REPAIR

In [37]:
training_history

,Iteration,Fitness,Accuracy,ExecutionTime,Discrete,Real
0,0,2,0,0.000530,"[1, 0, 0]","[28.265234033663333, 144.13711249278165, 83.50..."
1,1,2,0,0.001839,"[1, 0, 0]","[28.265234033663333, 144.13711249278165, 83.50..."
2,2,2,0,0.003328,"[1, 0, 0]","[28.265234033663333, 144.13711249278165, 83.50..."
3,3,2,0,0.004679,"[1, 0, 0]","[28.265234033663333, 144.13711249278165, 83.50..."
4,4,2,0,0.005857,"[1, 0, 0]","[28.265234033663333, 144.13711249278165, 83.50..."
...,...,...,...,...,...,...
295,295,2,0,0.222302,"[1, 0, 0]","[28.265234033663333, 144.13711249278165, 83.50..."
296,296,2,0,0.222895,"[1, 0, 0]","[28.265234033663333, 144.13711249278165, 83.50..."
297,297,2,0,0.223390,"[1, 0, 0]","[28.265234033663333, 144.13711249278165, 83.50..."
298,298,2,0,0.224405,"[1, 0, 0]","[28.265234033663333, 144.13711249278165, 83.50..."


In [38]:
discrete_solution = list(training_history["Discrete"].iloc[-1])
discrete_solution

[1, 0, 0]

In [39]:
schedule_solution = list(training_history["Real"].iloc[-1])
schedule_solution

[28.265234033663333,
 144.13711249278165,
 83.50079465030274,
 177.92764593895924,
 21.092480023102386,
 69.69110821655003,
 48.59062766013451,
 169.20715042110905,
 185.54524949155922,
 238.76209871923965,
 29.58134383110912,
 123.92296619969025,
 184.45870953289273,
 232.0870911540636]

In [40]:
update_schedule(schedule_solution)
schedule

{1: {'MAD': (28.265234033663333, 28.265234033663333),
  'BAR': (144.13711249278165, 83.50079465030274),
  'FIG': (177.92764593895924, 177.92764593895924)},
 2: {'MAD': (21.092480023102386, 21.092480023102386),
  'ZAR': (69.69110821655003, 48.59062766013451),
  'BAR': (169.20715042110905, 185.54524949155922),
  'FIG': (238.76209871923965, 238.76209871923965)},
 3: {'MAD': (29.58134383110912, 29.58134383110912),
  'BAR': (123.92296619969025, 184.45870953289273),
  'FIG': (232.0870911540636, 232.0870911540636)}}

In [41]:
is_feasible({"real": schedule_solution, "discrete": discrete_solution})

True

Servicios: 3, 5, 10, 20, 50, 100

Repeticiones: 30
Individuos: 5 - 20, step: 5
Iteraciones: 100 - 500, step: 100

Nº de trenes planificados. Media en las ejecuciones.
Horarios.
Desviación del tren planificado con respecto a la solicitado. Con la mejor solución en cada una de las repeticiones (30 mejores soluciones).

Modificación función objetivo con beneficio del gestor.

In [42]:
request = [0, 148, 152, 180, 8, 28, 30, 165, 167, 210, 30, 180, 182, 225]

In [43]:
schedule_solution

[28.265234033663333,
 144.13711249278165,
 83.50079465030274,
 177.92764593895924,
 21.092480023102386,
 69.69110821655003,
 48.59062766013451,
 169.20715042110905,
 185.54524949155922,
 238.76209871923965,
 29.58134383110912,
 123.92296619969025,
 184.45870953289273,
 232.0870911540636]

In [46]:
diffs = [abs(request[i] - schedule_solution[i]) for i in range(len(request))]
diffs

[28.265234033663333,
 3.862887507218346,
 68.49920534969726,
 2.0723540610407554,
 13.092480023102386,
 41.691108216550035,
 18.59062766013451,
 4.2071504211090485,
 18.54524949155922,
 28.762098719239646,
 0.4186561688908803,
 56.07703380030975,
 2.458709532892726,
 7.087091154063586]

In [54]:
total_diff = np.sum(diffs)  # Tantos valores como ejecuciones -> media y std
total_diff

293.6298861394715

In [52]:
schedule

{1: {'MAD': (28.265234033663333, 28.265234033663333),
  'BAR': (144.13711249278165, 83.50079465030274),
  'FIG': (177.92764593895924, 177.92764593895924)},
 2: {'MAD': (21.092480023102386, 21.092480023102386),
  'ZAR': (69.69110821655003, 48.59062766013451),
  'BAR': (169.20715042110905, 185.54524949155922),
  'FIG': (238.76209871923965, 238.76209871923965)},
 3: {'MAD': (29.58134383110912, 29.58134383110912),
  'BAR': (123.92296619969025, 184.45870953289273),
  'FIG': (232.0870911540636, 232.0870911540636)}}

In [56]:
# Discrepancia por estación.

stations = np.sum([len(service) for service in schedule.values()])
stations

10

In [57]:
total_diff / stations

29.362988613947152